<a href="https://colab.research.google.com/github/DavideBruni/CIDL_lab/blob/main/Lecture2/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torchvision


train_mnist = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

test_mnist = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

# **Exercise 0**

In [6]:
model = torch.nn.Sequential(
    torch.nn.Linear(28 * 28, 300),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(300, 300),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(300, 10),
    torch.nn.Softmax(dim=1)
)

model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=300, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=300, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

In [7]:
from tqdm import tqdm

dl = torch.utils.data.DataLoader(train_mnist, batch_size=32, shuffle=True)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 5

for epoch in range(5):
    bar = tqdm(dl)
    for digit, cls in bar:
        digit = digit.to("cuda")    #move to GPU!
        cls = cls.to("cuda")

        digit = digit.view(digit.shape[0], 28 * 28) #reshape!
        pred = model(digit)

        loss = loss_fn(pred, cls)
        accuracy = (pred.argmax(dim=1) == cls).float().mean()
        bar.set_description(f"Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Loss: 1.4919, Accuracy: 0.9688: 100%|██████████| 1875/1875 [00:23<00:00, 78.62it/s]


In [8]:
import torch.nn.functional as F

correct = 0
total = 0
model.eval()

test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=32, shuffle=True)
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to("cuda")    #move to GPU!
        labels = labels.to("cuda")

        images = images.view(images.shape[0], 28 * 28)    #reshape!
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')

Accuracy on the test set: 94.49%
